## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Cabo San Lucas,MX,22.8909,-109.9124,96.51,48,1,11.50,clear sky
1,1,College,US,64.8569,-147.8028,60.75,77,90,5.75,light rain
2,2,Barrow,US,71.2906,-156.7887,35.62,75,40,8.05,scattered clouds
3,3,Carnarvon,AU,-24.8667,113.6333,62.67,55,75,13.80,broken clouds
4,4,Atuona,PF,-9.8000,-139.0333,78.40,63,14,23.38,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
4,4,Atuona,PF,-9.8000,-139.0333,78.40,63,14,23.38,few clouds
7,7,Hilo,US,19.7297,-155.0900,85.28,62,1,10.36,clear sky
10,10,Labuhan,ID,-6.8844,112.2051,75.51,78,100,10.07,overcast clouds
11,11,Acapulco,MX,16.8634,-99.8901,87.62,66,75,8.05,broken clouds
13,13,Thai Binh,VN,20.4500,106.3333,86.00,78,85,8.57,overcast clouds
17,17,Banff,CA,51.1762,-115.5698,84.58,21,10,8.72,clear sky
23,23,Old Orchard Beach,US,43.5173,-70.3775,77.43,89,90,1.01,overcast clouds
29,29,Nizwa,OM,22.9333,57.5333,85.86,62,100,1.90,overcast clouds
31,31,Batticaloa,LK,7.7102,81.6924,81.91,75,100,4.29,overcast clouds
33,33,Ormara,PK,25.2088,64.6357,83.25,85,100,22.79,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID        249
City           249
Country        249
Lat            249
Lng            249
Max Temp       249
Humidity       249
Cloudiness     249
Wind Speed     249
Description    249
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(how="any")

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
4,Atuona,PF,78.40,few clouds,-9.8000,-139.0333,
7,Hilo,US,85.28,clear sky,19.7297,-155.0900,
10,Labuhan,ID,75.51,overcast clouds,-6.8844,112.2051,
11,Acapulco,MX,87.62,broken clouds,16.8634,-99.8901,
13,Thai Binh,VN,86.00,overcast clouds,20.4500,106.3333,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(how="any")

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Description</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))